In [11]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report,roc_auc_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from joblib import Parallel,delayed

def svar(X):
    n = float(len(X))
    svar=(sum([(x-np.mean(X))**2 for x in X]) / n)* n/(n-1.)
    return svar

def CronbachAlpha(itemscores):
    itemvars = [svar(item) for item in itemscores]
    tscores = [0] * len(itemscores[0])
    for item in itemscores:
        for i in range(len(item)):
            tscores[i]+= item[i]
    nitems = len(itemscores)
    Calpha=nitems/(nitems-1.) * (1-sum(itemvars)/ svar(tscores))
    return Calpha

from scipy.stats import iqr,skew,kurtosis,pearsonr
from scipy.stats import variation,moment

def get_features1(x):
    temp = []
    temp.append(x[-1])
    temp.append(x[0])
    temp.append(x[len(x)//2])
    temp.append(iqr(x))
    temp.append(skew(x))
    temp.append(kurtosis(x))
    temp.append(variation(x))
    temp.append(iqr(x[:len(x)//2]))
    temp.append(skew(x[:len(x)//2]))
    temp.append(kurtosis(x[:len(x)//2]))
    temp.append(variation(x[:len(x)//2]))
    temp.append(iqr(x[len(x)//2:]))
    temp.append(skew(x[len(x)//2:]))
    temp.append(kurtosis(x[len(x)//2:]))
    temp.append(variation(x[len(x)//2:]))
    return np.array(temp)

# def get_features()

def get_features(temp,user):
    if temp.shape[0]<10:
        return -1,-1,-1,-1,-1
    if temp['time'].values[-1]-temp['time'].values[0]>1000:
        return -1,-1,-1,-1,-1
    if np.isnan(temp['score'].values[-1]):
        return -1,-1,-1,-1,-1
    a = temp['stress_likelihood'].values
    b = temp['all_scores'].values[-1]
    c = user
    hour = datetime.fromtimestamp(temp['time'].values[0]).hour
    if hour<8:
        d = 0
    elif hour>8 and hour < 16:
        d = 1
    else:
        d = 2
    e = user+temp['day'].values[-1]
    return a,b,c,d,e

def get_features2(temp,user):
    if temp.shape[0]<10:
        return -1,-1,-1,-1,-1
    if temp['time'].values[-1]-temp['time'].values[0]>1000:
        return -1,-1,-1,-1,-1
    g = temp['score'].values
    if len(g[~np.isnan(g)])>0:
        return -1,-1,-1,-1,-1
#     if np.isnan(temp['score'].values[-1]):
#         return -1,-1,-1,-1,-1
    a = temp['stress_likelihood'].values
    b = temp['all_scores'].values[-1]
    c = user
    hour = datetime.fromtimestamp(temp['time'].values[0]).hour
    if hour<8:
        d = 0
    elif hour>8 and hour < 16:
        d = 1
    else:
        d = 2
    e = user+temp['day'].values[-1]
    return a,b,c,d,e

def get_user_data(user_data,user):
    X_date = []
    X = []
    y = []
    groups = []
    days = []
    if len(np.unique(user_data.dropna()['day'].values))<4:
        return [],[],[],[],[]
    user_data = user_data.sort_values('localtime').reset_index(drop=True)
    count = 0
    columns=['window', 'localtime', 'day', 'stress_likelihood', 'imputed',
       'likelihood_mean', 'all_scores', 'score', 'label', 'version', 'user',
       'time']
    count = 0
    data = Parallel(n_jobs=20,verbose=1)(delayed(get_features)(user_data.loc[i:i+10],user) for i,row in user_data.iterrows()
                                        if not np.isnan(user_data.loc[i:i+10]['score'].values[-1]))
    data1 = Parallel(n_jobs=20,verbose=1)(delayed(get_features2)(user_data.loc[i:i+10],user) for i,row in user_data.iterrows()
                                        if i%10==0)
    
    data = data+data1
    for a in data:
        if not isinstance(a[2], str):
            continue
        X.append(a[0])
        y.append(a[1])
        groups.append(a[2])
        X_date.append(a[3])
        days.append(a[4])
    print(len(X))
    return X,y,groups,X_date,days


def get_data(all_stress):
    X_date,X,y,groups,days = [],[],[],[],[]
    final = [get_user_data(all_stress[all_stress.user.isin([user])],user) 
                                          for user in np.unique(all_stress['user'].values)]
    for a in final:
        X.extend(a[0])
        y.extend(a[1])
        groups.extend(a[2])
        X_date.extend(a[3])
        days.extend(a[4])
    return X,y,groups,X_date,days

In [9]:
import warnings
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler,OneHotEncoder
all_stress_left = pickle.load(open('../data/stress_ema_md2k_aa_rice_left_ppg.p','rb'))
# all_stress_left = all_stress_left[all_stress_left.imputed.isin([0])]
all_stress_right = pickle.load(open('../data/stress_ema_md2k_aa_rice_right1_ppg.p','rb'))
# all_stress_right = all_stress_right[all_stress_right.imputed.isin([0])]

In [ ]:
X2,y2,groups2,X_date2,days2 = get_data(all_stress_left)
X3,y3,groups3,X_date3,days3 = get_data(all_stress_right)
Xf,yf,groupsf,X_datef,daysf = X2+X3,y2+y3,groups2+groups3,X_date2+X_date3,days2+days3

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   4 out of  10 | elapsed:    8.4s remaining:   12.6s
[Parallel(n_jobs=20)]: Done  10 out of  10 | elapsed:    8.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 440 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done 818 out of 818 | elapsed:    4.1s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


811


[Parallel(n_jobs=20)]: Done  18 out of  28 | elapsed:    6.4s remaining:    3.6s
[Parallel(n_jobs=20)]: Done  28 out of  28 | elapsed:    6.4s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 280 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    3.5s
[Parallel(n_jobs=20)]: Done 1288 out of 1327 | elapsed:    5.2s remaining:    0.2s
[Parallel(n_jobs=20)]: Done 1327 out of 1327 | elapsed:    5.2s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


1323


[Parallel(n_jobs=20)]: Done  24 out of  31 | elapsed:    2.8s remaining:    0.8s
[Parallel(n_jobs=20)]: Done  31 out of  31 | elapsed:    2.8s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 280 tasks      | elapsed:    1.4s
[Parallel(n_jobs=20)]: Done 780 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 1074 out of 1074 | elapsed:    3.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


...

[Parallel(n_jobs=20)]: Done  12 out of  12 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=20)]: Done  12 out of  12 | elapsed:    2.0s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 280 tasks      | elapsed:    1.3s
[Parallel(n_jobs=20)]: Done 808 out of 808 | elapsed:    2.8s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


804


[Parallel(n_jobs=20)]: Done   5 out of   8 | elapsed:    1.6s remaining:    1.0s
[Parallel(n_jobs=20)]: Done   8 out of   8 | elapsed:    1.6s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 280 tasks      | elapsed:    1.2s
[Parallel(n_jobs=20)]: Done 610 out of 649 | elapsed:    2.1s remaining:    0.1s
[Parallel(n_jobs=20)]: Done 649 out of 649 | elapsed:    2.2s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.


641


[Parallel(n_jobs=20)]: Done   8 out of  11 | elapsed:    5.8s remaining:    2.2s
[Parallel(n_jobs=20)]: Done  11 out of  11 | elapsed:    5.8s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:    0.4s
[Parallel(n_jobs=20)]: Done 286 tasks      | elapsed:    1.5s
[Parallel(n_jobs=20)]: Done 786 tasks      | elapsed:    3.3s


In [6]:
pickle.dump([Xf,yf,groupsf,X_datef,daysf],open('../data/semi_supervised_no_imputation.p','wb'))

In [7]:
from sklearn import linear_model
Xf,yf,groupsf,X_datef,daysf = pickle.load(open('../data/semi_supervised_no_imputation.p','rb'))
X,y1,groups1,X_date_final = np.array(Xf),np.array(yf),np.array(groupsf),OneHotEncoder().fit_transform(np.array(X_datef).reshape(-1,1))
Xfeatures = np.array([get_features1(x) for x in X])
print(Xfeatures.shape)

(3180, 15)


In [ ]:
print(confusion_matrix(all_y,all_y_pred),classification_report(all_y,all_y_pred))

In [ ]:
getf1(Xfeatures,labels,groups)
# y.shape

In [ ]:
from frameworks.CPLELearning

In [ ]:
X[0]